In [1]:
import re, os
import shutil
import pickle
from logerror import TestError
import pandas as pd

root_folder = "/mnt/hgfs/projlogs/SFW/sops1/SOPS/Performance Test - Performance_Test1"
root_folder = "/mnt/hgfs/projlogs/SFW/hq1/Performance Test - Performance_Test1"
# MILL, POND, OPP1, OPP2, FOX
root_folder = "/mnt/hgfs/projlogs/PSU/POND"

test_folder = root_folder #+ '/' + location
files = [ f for f in os.listdir(test_folder) if os.path.isfile(os.path.join(test_folder,f)) and f.find('.log.')>0 ]
# print('preliminary files:', files)
files = [ f for f in files if f.find('lck')==-1 ]

# print('for ', location, 'found log files: ', files)
print('found', len(files), 'log files: ', files)

found 2 log files:  ['PONDF1OC_INJPOND_0.log.1', 'PONDF1OC_INJPOND_0.log.2']


In [2]:
# FNAME = "LocalInjectorCOBAPPSTS03_0.log.1"

if len(files)==1:
    FNAME = files[0]
elif len(files)>1:
    print('multiple files - merging into _log.all')
    FNAME = '_log.all'
    with open(os.path.join(test_folder, FNAME), 'wb') as wfd:
        for file in files:
            with open(os.path.join(test_folder, file),'rb') as fd:
                shutil.copyfileobj(fd, wfd, 1024*1024*100)
                #100MB per writing chunk to avoid reading big file into memory.

else:
    if not FNAME:
        print('no files found - please assign FNAME manually')
        exit(1)
    

multiple files - merging into _log.all


In [3]:
print('processing file: ', test_folder + '/' + FNAME)

with open(test_folder + '/' + FNAME,'r') as f:
    alllog = f.read()

errorlog = re.findall(r'(?<=\n).+?\[SEVERE\].+?id \d+?,[\w\s]+?\):.+?(?=\n)', alllog)
print('severe count:', len(errorlog))

processing file:  /mnt/hgfs/projlogs/PSU/POND/_log.all
severe count: 69929


In [4]:
errors = []
vusers = {}
for err in errorlog:
    time = err[:20]
    usr = re.search(r'user.(\d+?),.cycle', err)
    user = usr.group(1)
    user = str(user).rjust(4,'0')
    cyc = re.search(r'cycle.(\d+?),.step', err)
    cycle = cyc.group(1)
    stp = re.search(r'step\s+?(.+?),\s+?id', err)
    step = stp.group(1)
    errtype = re.search(r'status.(.+?)\)', err)
    error_type = errtype.group(1)
    message = err[err.find('):'):]
    new_record = TestError(time, user, cycle, step, error_type, message, test_folder)
    errors.append(new_record)
    if user in vusers.keys():
        vusers[user].append(new_record)
    else:
        vusers[user] = [new_record]

print('errors:', len(errors))
pickle.dump((errors, vusers), open(os.path.join(test_folder, 'errors.pickle'), 'wb' ))

errors: 69929


In [ ]:
# errors, vusers = pickle.load(open(os.path.join(test_folder, 'errors.pickle'), 'rb'))

In [5]:
sorted_keys = sorted(vusers.keys())
print(">> user, i, e.time, e.cycleid, e.errortype, etype, ")
print('erroring users', len(sorted_keys))
for user in sorted_keys:
    print('\nuser ' + user + ' err count:', len(vusers[user]))
    prior_cyc_index = -1
    consecutive_count = 0
    consecutivness = ''
    for i in range(len(vusers[user])-1,-1,-1):
        e = vusers[user][i]
        try:
            etype = e.validation
        except AttributeError:
            try:
                etype = e.dditem
            except AttributeError:
                etype = '!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!'

        # print('pre: cyc', e.cycleid, ' prior', prior_cyc_index, ' consecutive_count', consecutive_count, 'consecutivness', consecutivness )
        # determine consecutivness
        if prior_cyc_index == -1:
            prior_cyc_index = int(e.cycleid)
        else:
            if prior_cyc_index - int(e.cycleid) <= 1 and i != 0:
                consecutive_count += 1
                prior_cyc_index = int(e.cycleid)
            else:
                consecutivness = ';' + str(e.cycleid) + '-' + str(int(e.cycleid) + consecutive_count) + consecutivness
                consecutive_count = 0
                prior_cyc_index = int(e.cycleid)
        cause = e.determine_casuse()
    if len(consecutivness):
        print('~~', consecutivness[1:])

print(end='\n')
print('8'*80)
print("e.time, user, e.cycleid, e.errortype, e.cause, usr err count")

pickle.dump((errors, vusers), open(os.path.join(test_folder, 'errors2.pickle'), 'wb' ))

>> user, i, e.time, e.cycleid, e.errortype, etype, 
erroring users 24

user 0000 err count: 2
~~ 1-1

user 0001 err count: 3
~~ 5-5;5-5

user 0002 err count: 5
~~ 2-3;3-4

user 0003 err count: 3
~~ 0-1

user 0004 err count: 125
~~ 2-2;3-4;8-128

user 0005 err count: 2
~~ 2-2

user 0006 err count: 2
~~ 8-8

user 0007 err count: 2
~~ 2-2

user 0009 err count: 540
~~ 18-556

user 0010 err count: 124
~~ 20-142

user 0011 err count: 768
~~ 4-770

user 0012 err count: 2
~~ 7-7

user 0013 err count: 15375
~~ 6-6;7-15379

user 0014 err count: 13
~~ 6-17

user 0015 err count: 37
~~ 1-36

user 0016 err count: 16207
~~ 5-16210

user 0018 err count: 18044
~~ 1-2;8-8;18-18057

user 0019 err count: 17975
~~ 0-17973

user 0020 err count: 20
~~ 0-1;4-5;14-14;17-22;23-24;26-27;28-28;32-32;34-34;36-36

user 0021 err count: 667
~~ 3-3;7-7;9-9;11-673

user 0022 err count: 4
~~ 1-1;1-1;4-4

user 0023 err count: 1

user 0024 err count: 7
~~ 29-29;32-33;35-35;55-55;63-63

user 0025 err count: 1

888888888888

In [ ]:
errors, vusers = pickle.load(open(os.path.join(test_folder, 'errors2.pickle'), 'rb'))

In [6]:
causes = {}

accu = []

for user in sorted_keys:
    e = vusers[user][0]
    msg = ''
    if e.cause in ['Warning Message', 'Critical Message', 'Question Message', 'dynamic response']:
        msg = e.cause_message
    accu.append([pd.to_datetime(e.time, infer_datetime_format=True), user,
                e.cycleid, e.errortype, e.cause, len(vusers[user]), msg])
    
    if e.cause in causes.keys():
        causes[e.cause] += 1
    else:
        causes[e.cause] = 1

print('count of erroring users:', len(sorted_keys))
for k,v in causes.items():
    print('cause', k, 'has count', v)

data = pd.DataFrame(accu, columns=['time', 'user', 'cycle', 'err_class', 'err_cause', 'err_count', 'message'])

count of erroring users: 24
cause Incorrect Response has count 14
cause Critical Message has count 4
cause Maximo forcefully signed out the user has count 3
cause Long Op has count 1
cause operation resulted in a table with zero rows has count 1
cause unknown validation fail has count 1


In [7]:
pd.options.display.max_rows = 999
pd.options.display.max_colwidth = 999
data

,time,user,cycle,err_class,err_cause,err_count,message
0,2018-01-30 10:33:21,0000,1,Incorrect Response,Incorrect Response,2,
1,2018-01-30 10:59:45,0001,5,Incorrect Response,Incorrect Response,3,
2,2018-01-30 11:01:59,0002,2,Incorrect Response,Incorrect Response,5,
3,2018-01-30 11:02:49,0003,0,Incorrect Response,Incorrect Response,3,
4,2018-01-30 10:38:37,0004,2,Incorrect Response,Incorrect Response,125,
5,2018-01-30 10:50:17,0005,2,Incorrect Response,Incorrect Response,2,
6,2018-01-30 11:50:44,0006,8,Success Validation Failure,Critical Message,2,BMXZZ1030E - GL Account 0603336 UP is only allowed on the worktype JOB. More information
7,2018-01-30 11:16:59,0007,2,Incorrect Response,Incorrect Response,2,
8,2018-01-30 11:28:44,0009,18,Success Validation Failure,Maximo forcefully signed out the user,540,
9,2018-01-30 11:51:28,0010,20,Success Validation Failure,Maximo forcefully signed out the user,124,
